### Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### Data

In [2]:
mnist_dataset, mnist_info = tfds.load('mnist', with_info=True, as_supervised=True)

Dl Completed...: 100%|██████████| 4/4 [00:02<00:00,  1.39 file/s]

Dataset mnist downloaded and prepared to /Users/amescua/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
# train, validation, test
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# We would like to scale our data in some way, to make the result more numerically stable (inputs between 0 and 1)
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [ ]:
# Shuffle and batch the data
BUFFER_SIZE = 10000 # we can not shuffle all the data at once, when we have a lot of data.

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE) # we shuffle 10000 at a time

# after shuffling, we can finally stract the data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Mini batch gradient descent to train our model
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE) # how many samples to take in each batch

# here we don't backpropagate, we don't need to batch, but our model expects the data in batch format too.
validation_data = validation_data.batch(num_validation_samples) 
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [ ]:
# Outline the model

input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),

                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),# hidden layers
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),

                            tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [ ]:
# Choose the optimizer and the loss function

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])